[![](./media/Db2_header_3.png)](https://www.ibm.com/demos/collection/db2-database/)
<a id="top">

In [41]:
%run refresh.ipynb

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Installing Db2 on OpenShift with MPP

This lab will take the user through the steps required to  deploy Db2 on an OpenShift Image in Db2 MPP Topography. This is excellent for Data Warehousing and ODS deployments. The requirement for this lab is that you have completed the installing Db2 on OpenShift lab previous to running this lab, as it creates the Db2 project and sets up the required environment leading up to the installation of Db2.

### How to Copy Code and Examples
Throughout this lab there are code samples that need to be copied and modified in a text editor. Any commands that need to be executed from a command line are found in grey boxes (an example is found below) has been designed to be easily copied.

In [42]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
""<pre id=112 onmousedown="window.clipline(112)" onmouseup="window.reset(112)  ">
Sample commands are found in cells like this.
</pre>
</div>


The entire contents of the text in the cell will be automatically copied when you click on the cell. The color of the background will change color briefly to indicate that the copy has completed. To paste commands into a terminal window, use the key combination **Control-Shift-v**. 

It may be easier to keep a terminal window on top of the Jupyter notebook when running these commands. When you have a terminal window displayed, right click on the title bar and select `Always on Top` to keep the screen visible during the duration of the lab.

### Open a Command Line
click the below button to get started !

In [43]:
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Command Line",button_style='primary')
output = widgets.Output()
display(button, output)
def on_button_clicked(b):
        ! gnome-terminal --window-with-profile=db2pot --working-directory=/home/db2pot
button.on_click(on_button_clicked)

Button(button_style='primary', description='Command Line', style=ButtonStyle())

Output()

## Passwords for this lab
### Operating system login & Web App Login
        UserID: db2pot
        password: 123qwe123

### Openshift Login 
        UserID: admin
        password: redhat

### Db2 Login
        UserID: db2inst1
        password: db2Inst1

## Setup Environment for MPP

First we have to remove our OLTP environment. Please follow the steps in order otherwise you may create issues with your OpenShift cluster.

Login as admin using password "redhat" using the below command and complete the following steps

In [44]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2.1 onmousedown="window.clipline(2.1)" onmouseup="window.reset(2.1)">
oc login -u admin -n db2

In [45]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2 onmousedown="window.clipline(2)" onmouseup="window.reset(2)">
helm delete db2u --purge

Monitor the Db2 pod deletion until there are no pods left running for Db2

In [46]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2.2 onmousedown="window.clipline(2.2)" onmouseup="window.reset(2.2)">
oc get pods

Delete the Db Persistent volume and Claim and then the db2vol directory we created in the previous lab

In [47]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2.4 onmousedown="window.clipline(2.4)" onmouseup="window.reset(2.4)">
oc delete pvc db2pvc

In [48]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2.5 onmousedown="window.clipline(2.5)" onmouseup="window.reset(2.5)">
oc delete pv db2-vol

In [49]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2.3 onmousedown="window.clipline(2.3)" onmouseup="window.reset(2.3)">
sudo rm -rf /data/db2vol1

### Creating the DB2 Environment for MPP

The second step will be to create the directory with the right permissions for our Db2s persistent volume (PV) and volume claim (PVC) to be built.
Db2 Supports host path file systems including local and attached storage as well as Cloud or shared storage such as NFS, PortWorks etc.

In [50]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=3 onmousedown="window.clipline(3)" onmouseup=" window.reset(3)">
sudo mkdir -p /data/db2vol1

In [51]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=4 onmousedown="window.clipline(4)" onmouseup="window.reset(4)">
sudo chmod 777 -R /data/db2vol1

In [52]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=5 onmousedown="window.clipline(5)" onmouseup="window.reset(5)">
sudo chgrp root -R /data/db2vol1


In [53]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=6.1 onmousedown="window.clipline(6.1)" onmouseup="  window.reset(6.1)">
sudo restorecon -Rv /data/db2vol1

In [54]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black;" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=7 onmousedown="window.clipline(7)" onmouseup="window.reset(7)">
sudo semanage fcontext -a -t container_file_t "/data/db2vol1(/.*)?"

### Create the Db2 Persistent Volume and Volume Claim

We can now re-use the file to create the Persistent Volume and Claim we used in the previous lab.

In [55]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=17 onmousedown="window.clipline(17)" onmouseup="window.reset(17)">
oc apply -f /home/db2pot/notebooks/db2vol.yaml

In [36]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=1711 onmousedown="window.clipline(1711)" onmouseup="window.reset(1711)">
cd /home/db2pot/charts/stable/ibm-db2warehouse/ibm_cloud_pak/pak_extensions

In [35]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=172 onmousedown="window.clipline(172)" onmouseup="window.reset(172)">
./pre-install/clusterAdministration/createSecurityClusterPrereqs.sh

In [36]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=173 onmousedown="window.clipline(173)" onmouseup="window.reset(173)">
./pre-install/namespaceAdministration/createSecurityNamespacePrereqs.sh db2

### Now Lets Install Db2 !

First we are going to move to the db2 deployment directory then run the Db2 install with the required parameters for a small install

In [56]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=18 onmousedown="window.clipline(18)" onmouseup=" window.reset(18)">
cd /home/db2pot/charts/stable/ibm-db2warehouse/ibm_cloud_pak/pak_extensions/common

## Update the "helm_options" File
We want to deploy a 4 Data Partion Single Node Db2 environment.To do this we need to pass to the database installation command a configuration file. This is not necessary for a single partition install, only if you wish to deploy mpp. As we are creating a options file we will also need to place storage settings in the file as well.
in the above directory create a file called "my_options" and the following settings

storage.existingClaimName="db2pvc"   
mln.total="4"    
images.db2u.replicas="1"   





In [57]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=19 onmousedown="window.clipline(19)" onmouseup="window.reset(19)">
./db2u-install --db-type db2wh --namespace db2 --release-name db2u  --tiller-namespace tiller --memory-size 18Gi --helm-opt-file ./my_options

The above installs a Db2 database with configurations for MPP Warehouse, in our Namespace / Project Db2, using our defined Db2 Persistent volume claim and sets the instance to a single node "replica" and 4 Data Partitions. the memory and CPU are not fixed limited

### Now we are going to monitor the deployments Progress

First of all we are going to get the status of the pods, you will have to run this command multiple times as it will take a few minutes for the install to download the containers and install them in your project.

In [58]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=20 onmousedown="window.clipline(20)" onmouseup="window.reset(20)">
oc get pods

When your installation is complete you should see the following status with the db2u-db2u-0 pod status at Running and the ready status as 1/1

If you want to just get incremental updates of changes to the pods status you can try running "oc get pods -w". This command will give you the current status at the time of running and any incremental changes in status.

The last pod status change you will see is the `db2u-db2u-restore-morph` pod/job status will be `Completed`. Ctrl-C to quit out at that point.

## Monitoring Activities from the Openshift Console
You can get an overview of the activities within the cluster by using the Openshift dashboard.

Click on here and login as **user:admin / pwd:redhat** https://localhost:8443/login

Once in the console, Navigate to the project Db2, click on applications then select pods to view pod status and browse from there.

[![](./media/Openshift.png)](https://localhost:8443/login)
<a id="top">

## Finding our what address to connect to Db2 on

Lets find out the ports that we can connect to db2 on, first run the following command

In [59]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=21 onmousedown="window.clipline(21)" onmouseup="  window.reset(21)">
oc get svc
</div>

```
NAME                 TYPE        CLUSTER-IP       EXTERNAL-IP   PORT(S)                                   AGE
db2u-db2u            ClusterIP   172.30.96.174    <none>        50000/TCP,50001/TCP,25000/TCP,25001/TCP,…  2m
db2u-db2u-engn-svc   NodePort    172.30.169.115   <none>        50000:31020/TCP,50001:31578/TCP            2m
db2u-db2u-internal   ClusterIP   None             <none>        50000/TCP,9443/TCP                         2m
db2u-db2u-ldap       NodePort    172.30.122.233   <none>        389:30154/TCP                              2m
db2u-etcd            ClusterIP   None             <none>        2380/TCP,2379/TCP                          2m
```

 Db2 exposes its service externally via the db2u-db2u-engn-svc service. In this case, 31020 is the non-SSL port Db2 will be listening on, and 31578 is the port that will support SSL connectivity. They are mapped to the db2u-db2u-0 50000/50001 ports that Db2 operates on.

To get your IP address for your server for external communication, use the following (if you are running on VMware):

In [60]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=22 onmousedown="window.clipline(22)" onmouseup=" window.reset(22)">
ip addr show | grep 'scope global noprefixroute dynamic'

In this example this is what it shows, you can use the inet address to connect to Db2.

```
result: inet 192.168.154.132/24 brd 192.168.154.255 scope global noprefixroute dynamic ens33
```

## Lets Connect !

The following commands are run using the Jupyter notebook. Select each cell and hit **Shift-Enter** keys together or chose the run button at the top of your screen to execute the code. Alternatively you can place your cursor on the grey sell and select the Run button at the top of the screen.

In [ ]:
%run db2.ipynb

In [ ]:
%sql connect to bludb user db2inst1 using redhat host <HOST IP>  port <PORT> 

If the connection is not successful, it may be due to Db2 not having started completely. You may need to wait for a few minutes for Db2 to start accepting connections. Once you do have a connection, you need to add some sample data to the database. The BLUDB database does not have the sample tables in it, so issue the following command to populate the EMPLOYEE and DEPARTMENT tables.

Note if you run TOP in your command window, as we are running inside the Openshift node you will see the db2sycs processes running. 1 per data partition deployed.

### Lets ask Db2 how many Data Partitions it has created

In [ ]:
%sql select * from syscat.DBPARTITIONGROUPDEF

As you can see we have 4 data partitions, and only one partition group defined. these all ruin on the single node we are deployed on. In a larger cluster they can be spread out across multiple nodes.

### Lets Load some Data in to Test our Database

we are going to create a market basket data and insert some data in via python data generator running on the notebook.

In [ ]:
%sql drop table market_basket if Exists

In [ ]:
%%sql 
create table market_basket (    
    Receipt_id INTEGER Not Null,
    TX_Date    DATE Not Null,
    TX_NO      INTEGER not Null,
    p_no        integer not null,
    PRODUCT    VARCHAR(60) not Null,
    QTY        Integer Not Null, 
  iTEM_COST    DECIMAL(15,2), primary key(receipt_id,tx_no,product)) organize by row

### Generate Our Data

In [ ]:
import csv
import ipywidgets as widgets
from IPython.display import display
button = widgets.Button(description="Generate Data",button_style='primary')
output = widgets.Output()
display(button, output)
def on_button_clicked(b):
 print('Data load starting')
 i=1
 e=0
 with open('basket_data.csv', newline='\n') as csvfile:
  reader = csv.DictReader(csvfile)
  for row in reader:
    if i == 1:
        insert_st = "insert into market_basket values( "+row['RECEIPT_ID']+",'"+row['TX_DATE']+"',"+row['TX_NO']+","+row['PRODUCT_ID']+",'"+row['PRODUCT']+"',"+row['QUANTITY']+","+row['ITEM_COST']+")"
    if i > 1:
        insert_st = insert_st = insert_st + ",( "+row['RECEIPT_ID']+",'"+row['TX_DATE']+"',"+row['TX_NO']+","+row['PRODUCT_ID']+",'"+row['PRODUCT']+"',"+row['QUANTITY']+","+row['ITEM_COST']+")"
    if i >=10 :
        %sql  -q {insert_st}
        insert_st=''
        i=0
    e= e+1
    i=i+1
    if e>=18000:
      break
 print('Data load finished')
 print ('Rows Inserted:')
 print(e)
button.on_click(on_button_clicked)

At this point you can try various SQL commands since we have established a connection to Db2.

In [ ]:
%sql select count(*) from market_basket

In [ ]:
%sql -pie SELECT product,sum(qty*item_cost) Revenue FROM market_basket group by product order by revenue desc fetch first 10 rows only 

# Using External tables
We now will take a backup of the table using External Tables. We are going to dump the table contents to our Scratch directory
first we will assign the scratch directory as the external table directory... Scratch is a directory defined for loading data to and from the system

In [88]:
%sql CALL SYSPROC.ADMIN_CMD( 'UPDATE DB CONFIG FOR BLUDB USING EXTBL_LOCATION "/mnt/blumeta0/scratch" IMMEDIATE')

In [ ]:
%%sql 
CREATE EXTERNAL TABLE '/mnt/blumeta0/scratch/market.csv' USING ( DELIMITER '|' NOLOG TRUE ) 
  AS SELECT * FROM  market_basket

In [ ]:
%sql create external table '/mnt/bludata0/scratch/test.tbl' as select * from market_basket

In [31]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=229 onmousedown="window.clipline(229)" onmouseup=" window.reset(229)">
oc rsh $(oc get pods | grep db2u-db2u-tools | cut -d ' ' -f1)

Now lets see if the tables result set has been dumped to disk.

In [32]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=2299 onmousedown="window.clipline(2299)" onmouseup=" window.reset(2299)">
ls /mnt/blumeta0/scratch/

# Lets get the file !

Using Rsync we can now retrieve the file back to our source system and you can browse through the file on your desktop

In [92]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=22999 onmousedown="window.clipline(22999)" onmouseup=" window.reset(22999)">
oc rsync --strategy=tar    $(oc get pods | grep db2u-db2u-tools | cut -d ' ' -f1):/mnt/blumeta0/scratch/ /home/db2pot/Desktop

## Other Interesting Stuff !

There are a range of different commands you can explore to learn about your Db2 Environment.

View the logs of the Db2 pod (or exchange Db2 pod name for any other).

In [69]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=30 onmousedown="window.clipline(30)" onmouseup=" window.reset(30)">
oc logs db2u-db2u-0

Describe the current Db2 pod status.

In [70]:
%%html
<div style="margin-left: 35px; border-style: solid; border-width: 1px; padding: 10px;background-Color:black" >
<p style=" color:white ;font-family:courier;background-Color:black"
<pre id=31 onmousedown="window.clipline(31)" onmouseup=" window.reset(31)">
oc describe pod db2u-db2u-0

[![](./media/db2_dte_footer.png)](https://www.ibm.com/demos/collection/db2-database/)
<a id="top">

#### Credits: IBM 2020, Phil Downey [phil.downey1@ibm.com], George Baklarz [baklarz@ca.ibm.com]